# Módulo 4 — IA Generativa para TI (BCN / SISRED)
## Notebook: Prompt Engineering aplicado a un caso SISRED + Visión (imagen)

**Objetivo del notebook (práctico y realista):**
1. Aplicar *Prompt Engineering* (Rol → Contexto → Tarea → Restricciones → Formato).
2. Construir un *asistente de triage* para SISRED: clasificar solicitudes y proponer acciones.
3. Probar una celda multimodal: **subir una imagen** + prompt: *“defíneme qué está pasando en la imagen”*.


## 0) Preparación

### Requisitos
- Python 3.9+
- `pip install openai pillow numpy pandas`

### API Key
Define tu variable de entorno en tu terminal antes de ejecutar:
- Windows (PowerShell): `setx OPENAI_API_KEY "TU_KEY"`
- macOS/Linux: `export OPENAI_API_KEY="TU_KEY"`


## 0) Preparación 

### Requisitos
- `pip install openai pillow numpy pandas`

### API Key
Define tu variable de entorno en tu terminal antes de ejecutar:
- Windows (PowerShell): `setx OPENAI_API_KEY "TU_KEY"`
- macOS/Linux: `export OPENAI_API_KEY="TU_KEY"`


In [3]:

%pip install -q openai

import os
import json
import base64
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

from openai import OpenAI

# API Key: usa la ya definida en el notebook si existe; si no, toma la variable de entorno
if "OPENAI_API_KEY" not in globals() or not OPENAI_API_KEY:
	OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Cliente (SDK nuevo usa api_key=... o lee de la env automáticamente)
client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else OpenAI()

# Modelos (ajusta si tu org lo requiere)
MODEL_TEXT = "gpt-4.1-mini"
MODEL_VISION = "gpt-4.1-mini"

print("OK: librerías cargadas. API key detectada:", bool(OPENAI_API_KEY))


Note: you may need to restart the kernel to use updated packages.
OK: librerías cargadas. API key detectada: True


## 1) Plantilla de Prompt Engineering (5 minutos)

Vamos a estandarizar la forma de pedirle cosas al modelo.

**Estructura:**
1) Rol  
2) Contexto (institucional)  
3) Tarea (qué queremos)  
4) Formato de salida (JSON / bullets / etc.)


In [4]:
def build_prompt(role: str, context: str, task: str, constraints: str, output_format: str) -> str:
    return f"""ROL:
{role}

CONTEXTO:
{context}

TAREA:
{task}

RESTRICCIONES:
{constraints}

FORMATO DE SALIDA:
{output_format}
""".strip()

SISRED_CONTEXT = """Estás asistiendo a un equipo TI/gestión documental de SISRED (Biblioteca del Congreso Nacional de Chile).
El objetivo es apoyar la gestión (triage, priorización y derivación), NO reemplazar el criterio humano.
""".strip()


## 2) Caso SISRED: “Triage inteligente” de solicitudes (≈15 minutos)

**Problema típico SISRED:** llegan solicitudes por correo/ticket/chat (texto libre).  
Queremos que el modelo ayude a:

- Clasificar la solicitud en una categoría operativa
- Estimar urgencia
- Proponer derivación (área responsable)
- Sugerir próximos pasos
- Entregar todo en formato **JSON** (integrable a un sistema)

> En producción: controles, logging, revisión humana y seguridad.


In [5]:
CATEGORIES = [
    "Administrativo",
    "Jurídico/Normativa",
    "TI/Incidentes",
    "Transparencia"
]

ROLE = "Eres un asistente institucional de triage para SISRED."
CONSTRAINTS = """- No inventes normativa ni datos.
- Si faltan datos, indícalo explícitamente.
- No prometas acciones: solo sugiere pasos recomendados.
- Mantén tono institucional y conciso.
""".strip()

OUTPUT_FORMAT = """Responde SOLO en JSON válido con las siguientes claves:
{
  "categoria": "<una de: Administrativo | Jurídico/Normativa | TI/Incidentes | Transparencia>",
  "urgencia": "<baja|media|alta>",
  "razon_breve": "<1-2 frases>",
  "area_sugerida": "<finanzas|administración|jurídica|TI|archivo|transparencia|otra>",
  "acciones_recomendadas": ["...", "...", "..."],
  "riesgos_si_no_se_atiene": ["...", "..."]
}
""".strip()

def call_llm_json(user_text: str) -> dict:
    task = f"""Clasifica y analiza la siguiente solicitud SISRED:

SOLICITUD:
\"\"\"{user_text}\"\"\"
""".strip()

    prompt = build_prompt(
        role=ROLE,
        context=SISRED_CONTEXT,
        task=task,
        constraints=CONSTRAINTS,
        output_format=OUTPUT_FORMAT
    )

    resp = client.responses.create(
        model=MODEL_TEXT,
        input=[{
            "role": "user",
            "content": [{"type": "input_text", "text": prompt}]
        }],
    )

    raw = resp.output_text.strip()

    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        return {"_error": "El modelo no devolvió JSON válido", "_raw": raw}


In [6]:
example_request = "Incidente en SISRED: error 500 al exportar resultados. Usuarios reportan que falla desde las 09:30. Revisar logs del backend."
call_llm_json(example_request)


{'categoria': 'TI/Incidentes',
 'urgencia': 'alta',
 'razon_breve': 'El error 500 afecta la funcionalidad de exportar resultados, impidiendo operaciones críticas desde las 09:30.',
 'area_sugerida': 'TI',
 'acciones_recomendadas': ['Revisar logs del backend para identificar la causa del error 500.',
  'Verificar si hay despliegues recientes o cambios en el sistema que puedan haber originado el problema.',
  'Priorizar la corrección para restablecer la exportación de resultados lo antes posible.'],
 'riesgos_si_no_se_atiene': ['Interrupción prolongada de la funcionalidad puede afectar la productividad y confianza de los usuarios.',
  'Posible acumulación de datos sin exportar, dificultando procesos posteriores y reportes.']}

In [7]:
examples = [
    "Se solicita informe jurídico conforme a la Ley de Transparencia para revisar implicancias en contratos de proveedores.",
    "Incidente en SISRED: error 500 al realizar búsqueda. Revisar logs del backend y autenticación.",
    "Favor gestionar orden de compra para insumos y adjuntar memorándum para aprobación.",
    "Requerimiento de acceso a la información pública: presupuesto y gastos del último trimestre. Indicar plazos y responsables."
]

rows = []
for t in examples:
    out = call_llm_json(t)
    rows.append({
        "texto": t,
        "categoria": out.get("categoria"),
        "urgencia": out.get("urgencia"),
        "area_sugerida": out.get("area_sugerida"),
        "razon_breve": out.get("razon_breve")
    })

pd.DataFrame(rows)


,texto,categoria,urgencia,area_sugerida,razon_breve
0,Se solicita informe jurídico conforme a la Ley...,Jurídico/Normativa,media,jurídica,La solicitud requiere análisis jurídico para c...
1,Incidente en SISRED: error 500 al realizar bús...,TI/Incidentes,alta,TI,"Error 500 afecta la funcionalidad de búsqueda,..."
2,Favor gestionar orden de compra para insumos y...,None,None,None,None
3,Requerimiento de acceso a la información públi...,Transparencia,media,transparencia,Solicitud relacionada con acceso a información...


## 4) Mini-reflexión (2 minutos)

- Esto no “automatiza” SISRED por sí solo.
- Pero sí reduce tiempo en:
  - clasificar
  - proponer pasos
  - estandarizar salidas (JSON)

**Clave institucional:** revisión humana + trazabilidad.


---

# 🖼️ 5) Celda de IMAGEN + PROMPT (≈8–10 minutos)

Haremos una llamada multimodal:
- Subes una imagen a la carpeta del notebook (o indicas ruta local)
- Enviamos **imagen + texto** al modelo
- Prompt base: **“defíneme qué está pasando en la imagen”**

> Sugerencia: usa una captura de SISRED, un diagrama, un ticket, o un pantallazo con un error.


In [9]:
def image_to_data_url(image_path: str) -> str:
    path = Path(image_path)
    if not path.exists():
        raise FileNotFoundError(f"No existe: {path.resolve()}")

    ext = path.suffix.lower()
    mime = "image/png" if ext in [".png"] else "image/jpeg"  # jpg/jpeg por defecto

    b64 = base64.b64encode(path.read_bytes()).decode("utf-8")
    return f"data:{mime};base64,{b64}"

def describe_image_with_prompt(image_path: str, prompt_text: str) -> str:
    data_url = image_to_data_url(image_path)

    resp = client.responses.create(
        model=MODEL_VISION,
        input=[{
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt_text},
                {"type": "input_image", "image_url": data_url}
            ]
        }]
    )
    return resp.output_text.strip()

# --- USO ---
# 1) Pon una imagen en la misma carpeta del notebook o indica ruta completa.
# 2) Cambia el nombre aquí:
IMAGE_PATH = "mi_imagen.jpg"   # <-- cambia esto

PROMPT_IMG = "Defíneme qué está pasando en la imagen. Responde en 3-5 bullets y menciona cualquier riesgo o anomalía si la ves y si es posible dime que personas son las que aparecen."

# Ejecuta:
print(describe_image_with_prompt(IMAGE_PATH, PROMPT_IMG))

- La imagen muestra a dos hombres vestidos formalmente, dándose la mano y posando para la foto en un ambiente institucional.
- Ambos llevan pines con la bandera de Chile en la solapa, sugiriendo un contexto oficial chileno.
- En el fondo hay dos banderas chilenas que refuerzan el carácter oficial del encuentro.
- No se observa ningún riesgo ni anomalía en la imagen; parece ser una foto protocolaria durante una reunión o evento diplomático.
- No puedo identificar quiénes son estas personas basándome solo en la imagen.
